In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [8]:
fashion_mnist=keras.datasets.fashion_mnist

In [9]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [10]:
train_images=train_images/255.0
test_images=test_images/255.0

In [11]:
train_images[0].shape

(28, 28)

In [12]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [13]:
def build_model(hp):
  model=keras.Sequential([
            keras.layers.Conv2D(
                filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                activation='relu',
                input_shape=(28,28,1)
            ),
             keras.layers.Conv2D(
                filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                activation='relu',
            ),
            keras.layers.Flatten(),
             keras.layers.Dense(
                units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                activation='relu',
            ),

            keras.layers.Dense(10,activation='softmax')

  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output1',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output1/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output1/Mnist Fashion/tuner0.json


In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |112               |64                
conv_1_kernel     |5                 |3                 
conv_2_filter     |64                |32                
conv_2_kernel     |5                 |3                 
dense_1_units     |80                |48                
learning_rate     |0.001             |0.001             

Epoch 1/3
  90/1688 [>.............................] - ETA: 7:43 - loss: 0.8351 - accuracy: 0.6997

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()